Номер группы, ФИО: Билецкий АО

Загрузить данные о больных, с наличием рака `load_breast_cancer()`. 
Разбить данные в соотношении 7/3, random_state=12.

* Рассмотреть методы из лекции о линейных моделях. Оценить разброс характеристики качества для каждого из методов с параметрами по умолчанию, проведя кросс-валидацию по 5ти блокам на тренировочный выборке.
* Выбрать один из методов для дальнейшей работы, демонстрирующий лучшее качество
* Подключить недостающие пакеты, выполнить при необходимости дополнительные преобразования
* Для выбранного метода подобрать лучшие значения гиперпараметров
* Оценить качество настроенного классификатора на тестовой выбоке

In [128]:
import numpy as np
from sklearn import datasets
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from plot_region import *

In [129]:
dataset = load_breast_cancer()

In [130]:
X,y=dataset['data'],dataset['target']

In [145]:
X_train, X_test_fin, y_train, y_test_fin= train_test_split(X, y, test_size=0.1, random_state=12)

In [146]:
pipe=[]
pipe.append(Pipeline(steps=[             ('scaler', StandardScaler()),
                                    ('estimator', SVC(kernel='rbf'))])
           )

pipe.append(Pipeline(steps=[             ('scaler', StandardScaler()),
                                    ('estimator', LinearSVC())])
           )

pipe.append(Pipeline(steps=[             ('scaler', StandardScaler()),
                                    ('estimator', SGDClassifier())])
           )


kfold=KFold(n_splits=5, shuffle=True, random_state=0)

In [160]:
mean = []
accuracy=[]  
for i in range(3):
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        pipe[i].fit(X_train, y_train)
        y_pred = pipe[i].predict(X_test)
        mean.append(accuracy_score(y_test, y_pred).round(2))
    accuracy.append(np.array(mean).mean().round(4))

print(accuracy)


[0.976, 0.97, 0.968]


Лучший -- SVC

In [163]:
c=np.arange(0.1,10,0.3)
gamma=np.arange(0.01,0.5,0.03)
param_grid={'estimator__gamma': gamma,'estimator__C': c}
grid_search = GridSearchCV(pipe[0], param_grid,cv=kfold)
grid_search.fit(X_train, y_train)
grid_search.best_params_

{'estimator__C': 8.500000000000002, 'estimator__gamma': 0.01}

In [157]:
pipe_fin = Pipeline(steps=[             ('scaler', StandardScaler()),
                                    ('estimator', SVC( C=8.5, kernel='rbf', gamma=0.01))])

In [158]:
mean = []
accuracy=[]  

for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    pipe_fin.fit(X_train, y_train)
    y_pred = pipe_fin.predict(X_test)
    mean.append(accuracy_score(y_test, y_pred).round(2))
accuracy.append(np.array(mean).mean().round(4))

print(accuracy)

[0.978]


In [159]:
y_pred=pipe_fin.predict(X_test_fin) 
accuracy_score(y_test_fin,y_pred)

0.9824561403508771